Games:
- Tetris (325) X
- Super Mario Land (315) X
- Super Mario Land 2 (323) X
- Super Mario Land 3 (308) X
- Pokemon Red (289) X
- Pokemon Blue (247) X
- The Legend of Zelda (231) 
- Kirby's Dream Land (156) 
- Kirby's Dream Land 2 (196)
- Donkey Kong (313)
- Donkey Kong Land (244)

In [2]:
%cd ../gameboyprep/data/longplay_h5_files/

/teamspace/studios/gameboyprep/data/longplay_h5_files


In [3]:
import h5py
import math
from functools import reduce
from pathlib import Path

In [8]:
frame_counts = []
for file in Path('.').glob('*.h5'):
    with h5py.File(file.name, 'r') as f:
        frame_counts.append(f['video_frames'].attrs['total_frames'])

In [18]:
print(max(frame_counts))

3320057


In [1]:
shard_size = 1024
for count in frame_counts:
    if count%2 != 0:
        count -= 1
    print(f'before: {count}')
    print(f'remainder: {count % shard_size}')

NameError: name 'frame_counts' is not defined

In [13]:
reduce(math.gcd, frame_counts)

1